In [45]:
! git clone https://github.com/wchill/HMP_Dataset.git

fatal: destination path 'HMP_Dataset' already exists and is not an empty directory.


In [3]:
! ls

HMP_Dataset


In [4]:
! ls HMP_Dataset/

Brush_teeth	Drink_glass  Getup_bed	  Pour_water	 Use_telephone
Climb_stairs	Eat_meat     impdata.py   README.txt	 Walk
Comb_hair	Eat_soup     Liedown_bed  Sitdown_chair
Descend_stairs	final.py     MANUAL.txt   Standup_chair


In [5]:
! ls HMP_Dataset/Drink_glass/

Accelerometer-2011-03-24-10-07-02-drink_glass-f1.txt
Accelerometer-2011-03-24-10-16-02-drink_glass-f1.txt
Accelerometer-2011-03-24-10-46-25-drink_glass-f1.txt
Accelerometer-2011-03-24-11-14-00-drink_glass-f1.txt
Accelerometer-2011-03-24-13-09-29-drink_glass-f1.txt
Accelerometer-2011-03-24-13-17-06-drink_glass-f1.txt
Accelerometer-2011-03-24-13-31-22-drink_glass-f1.txt
Accelerometer-2011-03-24-16-08-10-drink_glass-f2.txt
Accelerometer-2011-03-24-16-08-29-drink_glass-f2.txt
Accelerometer-2011-04-05-18-55-33-drink_glass-f1.txt
Accelerometer-2011-04-08-17-33-35-drink_glass-f3.txt
Accelerometer-2011-04-08-17-35-00-drink_glass-f3.txt
Accelerometer-2011-04-08-18-10-09-drink_glass-m4.txt
Accelerometer-2011-04-11-12-55-39-drink_glass-f1.txt
Accelerometer-2011-04-11-13-17-55-drink_glass-f1.txt
Accelerometer-2011-04-12-21-43-48-drink_glass-m8.txt
Accelerometer-2011-04-12-21-44-55-drink_glass-m8.txt
Accelerometer-2011-05-30-21-06-15-drink_glass-f1.txt
Accelerometer-2011-05-30-21-06-42-drink_glass-

In [38]:
from pyspark.sql.types import StructType, StructField, IntegerType

schema = StructType([StructField("x",IntegerType(),True),
                     StructField("y",IntegerType(),True),
                     StructField("z",IntegerType(),True)])

In [46]:
import os
os.listdir()

['heapdump.20180817.215756.12210.0008.phd',
 'HMP_Dataset',
 'Snap.20180817.215735.12210.0004.trc',
 'Snap.20180817.215749.12210.0007.trc',
 'javacore.20180817.215735.12210.0003.txt',
 'Snap.20180817.215756.12210.0012.trc',
 'core.20180817.215735.12210.0001.dmp',
 'heapdump.20180817.215749.12210.0005.phd',
 'Snap.20180817.215756.12210.0013.trc',
 'javacore.20180817.215756.12210.0010.txt',
 'javacore.20180817.215749.12210.0006.txt',
 'core.9415',
 'javacore.20180817.215756.12210.0011.txt',
 'javacore.20180817.215839.12210.0015.txt',
 'Snap.20180817.215839.12210.0016.trc',
 'heapdump.20180817.215756.12210.0009.phd',
 'heapdump.20180817.215735.12210.0002.phd']

In [47]:
os.listdir("HMP_Dataset")

['README.txt',
 'Climb_stairs',
 'Getup_bed',
 'MANUAL.txt',
 'final.py',
 'Eat_soup',
 'Comb_hair',
 'Liedown_bed',
 'Sitdown_chair',
 'Eat_meat',
 'Descend_stairs',
 'Brush_teeth',
 'impdata.py',
 'Pour_water',
 'Walk',
 '.git',
 'Drink_glass',
 'Standup_chair',
 '.idea',
 'Use_telephone']

In [48]:
file_list = os.listdir('HMP_Dataset/')

In [49]:
file_list_filtered = [s for s in file_list if '_' in s]
file_list_filtered

['Climb_stairs',
 'Getup_bed',
 'Eat_soup',
 'Comb_hair',
 'Liedown_bed',
 'Sitdown_chair',
 'Eat_meat',
 'Descend_stairs',
 'Brush_teeth',
 'Pour_water',
 'Drink_glass',
 'Standup_chair',
 'Use_telephone']

In [50]:
df = None

from pyspark.sql.functions import lit

for category in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/' + category)
    
    for data_file in data_files:
        print(data_file)
        temp_df = spark.read.option('header','false').option('delimiter',' ').csv('HMP_Dataset/' + category + '/' + data_file,schema=schema)
        temp_df = temp_df.withColumn('Class',lit(category))
        temp_df = temp_df.withColumn('Source',lit(data_file))
        
        if df is None:
            df = temp_df
        else:
            df = df.union(temp_df)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45189)
Traceback (most recent call last):
  File "/usr/local/src/spark21master/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/src/spark21master/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Accelerometer-2011-05-30-20-54-39-climb_stairs-f1.txt


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:45189)

In [ ]:
df.show()

In [ ]:
from pyspark.ml.feature import StringIndexer
print('Imported StringIndexer')

indexer = StringIndexer(inputCol = 'Class',outputCol = 'ClassIndex')
indexed = indexer.fit(df).transform(df)
indexed.show()

In [ ]:
from pyspark.ml.feature import OneHotEncoder
print('Imported OneHotEncoder')

encoder = OneHotEncoder(inputCol='ClassIndex',outputCol='CategoryVec')
encoded = encoder.transform(indexed)

In [ ]:
encoded.show()

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
print('Imported VectorAssembler')

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
features_vectorized = vectorAssembler.transform(encoded)
features_vectorized.show()

In [ ]:
from pyspark.ml.feature import Nomalizer

normalizer = Normalizer(inputCol="features",outputCol = "features_normed",p=1.0)
features_normed = normalizer.transform(features_vectorized)
features_normed.show()

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer,encoder,vectorAssembler,normalizer])

model = pipeline.fit(df)
prediction = model.transform(df)
prediction.show()
